In [64]:
#Import arcpy and set workspace
import arcpy
from arcpy import env
env.workspace="H:/UEP 235/Final_Project/EQIPAnalysis/EQIPAnalysis.gdb"
env.addOutputsToMap = False

In [65]:
#Gathering state names for each county 
arcpy.management.MakeFeatureLayer(
    "H:/UEP 235/Final_Project/Federal Codes/Gazetteer_National_GDB/Gazetteer_National_GDB.gdb/Topical_Gazetteer/FederalCodes",
    "FederalCodesCounties",
    "Census_Class_Code LIKE 'H%' OR Census_Class_Code='C7'"
)
arcpy.conversion.ExportFeatures(
    in_features="FederalCodesCounties",
    out_features="H:/UEP 235/Final_Project/EQIPAnalysis/EQIPAnalysis.gdb/FederalCodes_Counties",
    use_field_alias_as_name="NOT_USE_ALIAS",
    field_mapping='feature_id "Feature_ID" true true false 4 Long 0 0,First,#,FederalCodesCounties,feature_id,-1,-1;feature_name "Feature_Name" true true false 120 Text 0 0,First,#,FederalCodesCounties,feature_name,0,120;feature_class "Feature_Class" true true false 50 Text 0 0,First,#,FederalCodesCounties,feature_class,0,50;state_name "State_Name" true true false 100 Text 0 0,First,#,FederalCodesCounties,state_name,0,100;state_numeric "State_Numeric" true true false 2 Text 0 0,First,#,FederalCodesCounties,state_numeric,0,2;county_name "County_Name" true true false 100 Text 0 0,First,#,FederalCodesCounties,county_name,0,100;county_numeric "County_Numeric" true true false 5 Text 0 0,First,#,FederalCodesCounties,county_numeric,0,5',
)
arcpy.management.CalculateField(
    "FederalCodes_Counties",
    "County_State_ID",
    "!state_numeric!!county_numeric!",
)

<Result 'H:/UEP 235/Final_Project/EQIPAnalysis/EQIPAnalysis.gdb\\FederalCodes_Counties'>

In [66]:
# Adding state names to county shapefile
arcpy.management.AddJoin(
    in_layer_or_view="AllCounties2022",
    in_field="GEOID",
    join_table="FederalCodes_Counties",
    join_field="County_State_ID",
    join_type="KEEP_COMMON",
    index_join_fields="INDEX_JOIN_FIELDS"
)
arcpy.management.CopyFeatures(
    "AllCounties2022_Layer",
    "H:/UEP 235/Final_Project/EQIPAnalysis/EQIPAnalysis.gdb/Boundary_Polygons/AllCountiesWStates"
)

<Result 'H:\\UEP 235\\Final_Project\\EQIPAnalysis\\EQIPAnalysis.gdb\\Boundary_Polygons\\AllCountiesWStates'>

In [67]:
#Extract contiguous US counties
arcpy.analysis.Select(
    "AllCountiesWStates",
    "H:/UEP 235/Final_Project/EQIPAnalysis/EQIPAnalysis.gdb/Boundary_Polygons/ContiguousUSCounties",
    "AllCounties2022_STATEFP NOT IN ('11', '69', '60', '66', '15', '02', '72', '78','!',' ')"
)

#Fix St. vs St issues
fc="ContiguousUSCounties_lyr"
fields=['AllCounties2022_NAME']
arcpy.management.MakeFeatureLayer(
    "ContiguousUSCounties",
    "ContiguousUSCounties_lyr"
)
arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="ContiguousUSCounties_lyr",
    selection_type="NEW_SELECTION",
    where_clause="AllCounties2022_NAME LIKE 'St.%'",
)
with arcpy.da.UpdateCursor(fc, fields) as cursor:
    for row in cursor:
        name=row[0]
        row[0] = name[0:2]+' '+name[4:]
        cursor.updateRow(row) 
arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="ContiguousUSCounties_lyr",
    selection_type="CLEAR_SELECTION",
)

#Fix 's vs. s issues
fc="ContiguousUSCounties_lyr"
fields=['AllCounties2022_NAME']
arcpy.management.MakeFeatureLayer(
    "ContiguousUSCounties",
    "ContiguousUSCounties_lyr"
)
arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="ContiguousUSCounties_lyr",
    selection_type="NEW_SELECTION",
    where_clause="AllCounties2022_NAME LIKE '%''s'",
)
with arcpy.da.UpdateCursor(fc, fields) as cursor:
    for row in cursor:
        name=row[0]
        row[0] = name[:-2]+name[-1:]
        cursor.updateRow(row) 
arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="ContiguousUSCounties_lyr",
    selection_type="CLEAR_SELECTION",
)

#Create unique text ID for states and counties together
arcpy.management.CalculateField(
    in_table="ContiguousUSCounties",
    field="County_States",
    expression='Concatenate($feature.AllCounties2022_NAME,", ",$feature.FederalCodes_Counties_state_name)',
    expression_type="ARCADE",
    code_block="""# Joins two or more fields into a single string value
                # More calculator examples at esriurl.com/CalculatorExamples
                def ConcatenateFields(*args):
                sep = "-"
                return sep.join([i for i in args if i])""",
    field_type="TEXT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)
arcpy.management.CalculateField(
    "EQIPContracts",
    "County_States",
    expression='Concatenate($feature.County,", ",$feature.State)',
    expression_type="ARCADE",
    code_block="""# Joins two or more fields into a single string value
                # More calculator examples at esriurl.com/CalculatorExamples
                def ConcatenateFields(*args):
                sep = "-"
                return sep.join([i for i in args if i])""",
    field_type="TEXT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

#Join EQIP contract data onto US counties
arcpy.management.AddJoin(
    in_layer_or_view="ContiguousUSCounties_lyr",
    in_field="County_States",
    join_table="EQIPContracts",
    join_field="County_States",
    join_type="KEEP_COMMON",
    index_join_fields="INDEX_JOIN_FIELDS"
)
arcpy.management.CopyFeatures(
    "ContiguousUSCounties_lyr",
    "H:/UEP 235/Final_Project/EQIPAnalysis/EQIPAnalysis.gdb/Boundary_Polygons/CountiesWContracts"
)

<Result 'H:\\UEP 235\\Final_Project\\EQIPAnalysis\\EQIPAnalysis.gdb\\Boundary_Polygons\\CountiesWContracts'>

In [68]:
#Create date field for space time cube
arcpy.management.ConvertTimeField(
    in_table="H:/UEP 235/Final_Project/EQIPAnalysis/EQIPAnalysis.gdb/Boundary_Polygons/CountiesWContracts",
    input_time_field="EQIPContracts_FY",
    input_time_format="yyyy",
    output_time_field="FiscalYear",
    output_time_type="DATE",
)

<Result 'H:/UEP 235/Final_Project/EQIPAnalysis/EQIPAnalysis.gdb/Boundary_Polygons/CountiesWContracts'>

In [69]:
#Filling in  suppressed values with zeros
arcpy.management.MakeFeatureLayer(
    "CountiesWContracts",
    "CountiesWContracts_lyr"
)
arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="CountiesWContracts_lyr",
    selection_type="NEW_SELECTION",
    where_clause="EQIPContracts_Suppressed = 'TRUE'",
)
arcpy.management.CalculateField(
    "CountiesWContracts_lyr",
    "EQIPContracts_Contract_count",
    "0",
)
arcpy.management.CalculateField(
    "CountiesWContracts_lyr",
    "EQIPContracts_Acres_Contracted",
    "0",
)
arcpy.management.CalculateField(
    "CountiesWContracts_lyr",
    "EQIPContracts_Dollars_Obligated",
    "0",
)
arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="CountiesWContracts_lyr",
    selection_type="CLEAR_SELECTION",
)

<Result 'CountiesWContracts_lyr'>

In [70]:
#Create space time cube
arcpy.stpm.CreateSpaceTimeCubeDefinedLocations(
    in_features="CountiesWContracts",
    output_cube="H:/UEP 235/Final_Project/EQIPAnalysis/EQIPContractsCube.nc",
    location_id="ContiguousUSCounties_AllCounties2022_GEOID",
    temporal_aggregation="NO_TEMPORAL_AGGREGATION",
    time_field="FiscalYear",
    time_step_interval="1 Years",
    time_step_alignment="END_TIME",
    reference_time=None,
    variables="EQIPContracts_Contract_count ZEROS;EQIPContracts_Acres_Contracted ZEROS;EQIPContracts_Dollars_Obligated ZEROS",
    summary_fields=None,
    in_related_table=None,
    related_location_id=None
)

<Result 'H:\\UEP 235\\Final_Project\\EQIPAnalysis\\EQIPContractsCube.nc'>

In [72]:
env.addOutputsToMap = True
arcpy.stpm.EmergingHotSpotAnalysis(
    "H:/UEP 235/Final_Project/EQIPAnalysis/EQIPContractsCube.nc",
    "EQIPCONTRACTS_CONTRACT_COUNT_NONE_ZEROS",
    "HotspotResults/Hotspots_contracts",
)
arcpy.stpm.EmergingHotSpotAnalysis(
    "H:/UEP 235/Final_Project/EQIPAnalysis/EQIPContractsCube.nc",
    "EQIPCONTRACTS_ACRES_CONTRACTED_NONE_ZEROS",
    "HotspotResults/Hotspots_acreage",
)
arcpy.stpm.EmergingHotSpotAnalysis(
    "H:/UEP 235/Final_Project/EQIPAnalysis/EQIPContractsCube.nc",
    "EQIPCONTRACTS_DOLLARS_OBLIGATED_NONE_ZEROS",
    "HotspotResults/Hotspots_dollars",
)

<Result 'H:/UEP 235/Final_Project/EQIPAnalysis/EQIPAnalysis.gdb\\HotspotResults\\Hotspots_dollars'>

In [73]:
#Add state boundaries
arcpy.management.MakeFeatureLayer(
    "Boundary_Polygons/States",
    "States_lyr"
)

<Result 'States_lyr'>

In [75]:
#Mean Center Calculations
arcpy.stats.MeanCenter(
    Input_Feature_Class=r"H:\UEP 235\Final_Project\EQIPAnalysis\EQIPAnalysis.gdb\Boundary_Polygons\CountiesWContracts",
    Output_Feature_Class=r"H:\UEP 235\Final_Project\EQIPAnalysis\EQIPAnalysis.gdb\MeanCenters\Acreage",
    Weight_Field="EQIPContracts_Acres_Contracted",
    Case_Field="EQIPContracts_FY",
)
arcpy.stats.MeanCenter(
    Input_Feature_Class=r"H:\UEP 235\Final_Project\EQIPAnalysis\EQIPAnalysis.gdb\Boundary_Polygons\CountiesWContracts",
    Output_Feature_Class=r"H:\UEP 235\Final_Project\EQIPAnalysis\EQIPAnalysis.gdb\MeanCenters\Dollars",
    Weight_Field="EQIPContracts_Dollars_Obligated",
    Case_Field="EQIPContracts_FY",
)
arcpy.stats.MeanCenter(
    Input_Feature_Class=r"H:\UEP 235\Final_Project\EQIPAnalysis\EQIPAnalysis.gdb\Boundary_Polygons\CountiesWContracts",
    Output_Feature_Class=r"H:\UEP 235\Final_Project\EQIPAnalysis\EQIPAnalysis.gdb\MeanCenters\Contracts",
    Weight_Field="EQIPContracts_Contract_count",
    Case_Field="EQIPContracts_FY",
)

<Result 'H:\\UEP 235\\Final_Project\\EQIPAnalysis\\EQIPAnalysis.gdb\\MeanCenters\\Contracts'>